# Radial Average
## Importing libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools
from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import henon_map as hm

from parameters import *

## Computing

In [ ]:
error_2 = {}
DA_2 = {}
count_matrix_2 = {}
average_matrix_2 = {}
raw_error_2 = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]

for epsilon in tqdm(epsilons, desc="Radial average"):
    # Extracting the radiuses with theta1 = theta2 = 0.0
    
    engine = hm.radial_scan.generate_instance(
        d_r, 
        alpha_values, 
        np.zeros(alpha_values.shape),
        np.zeros(alpha_values.shape),
        epsilon,
        starting_position=starting_position
    )
    all_radiuses = engine.compute(turn_sampling)
    
    # Computing of the 2D values
    
    skips = [1]
    while True:
        if (alpha_max_samples - 1) // skips[-1] > 4:
            skips.append(skips[-1] * 2)
        else:
            break
    
    for i in skips:
        alpha = alpha_values[::i]
        cutted_radiuses = all_radiuses[::i]
        value = integrate.simps(cutted_radiuses ** 2, alpha, axis=0)
        less_value = integrate.simps(cutted_radiuses[::2] ** 2, alpha[::2], axis=0)
        uncertainty = np.abs((value - less_value))
        
        DA = np.sqrt(value * 2 / np.pi)
        uncertainty = 0.5 * np.power(value * 2 / np.pi, -0.5) * uncertainty
        DA_5[(epsilon, cutted_radiuses.shape)] = np.asarray(DA)
        error_5[(epsilon, cutted_radiuses.shape)] = uncertainty 
    
    # Rest of the Angular Averaging process
    
    values = []
    for i in tqdm(range(len(turn_sampling))):
        temp_values = np.array([[]])
        for index, j in enumerate(range(0, samples, 128)):
            stopping = (j + 128 if j != samples - 1 else samples)
            radiuses = all_radiuses[j : stopping, i]

            engine = hm.full_track.generate_instance(
                radiuses,
                alpha_values[j : stopping],
                np.zeros(alpha_values.shape)[j : stopping],
                np.zeros(alpha_values.shape)[j : stopping],
                np.ones(alpha_values.shape, dtype=np.int)[j : stopping] * turn_sampling[i],
                epsilon)

            x, y, px, py = engine.compute()
            
            _ = engine.accumulate_and_return(n_subdivisions)
            tmp_count_matrix = engine.count_matrix
            tmp_avg_matrix = engine.matrices
            if index == 0:
                list_count_matrix = tmp_count_matrix
                list_avg_matrix = tmp_avg_matrix
            else:
                list_count_matrix = np.concatenate((list_count_matrix, tmp_count_matrix))
                list_avg_matrix = np.concatenate((list_avg_matrix, tmp_avg_matrix))
        
        count_total, matrix_total, result_total = hm.cpu_accumulate_and_return(list_count_matrix, list_avg_matrix)
        
        values.append(result_total)
        count_matrix_2[(epsilon, samples, turn_sampling[i])] = count_total
        average_matrix_2[(epsilon, samples, turn_sampling[i])] = matrix_total
    
    steps = [1]
    while True:
        if (values[0][0].shape[0] - 1) / steps[-1] > 4:
            steps.append(steps[-1] * 2)
        else:
            break

    for jump in steps:
        for j in range(len(values[0])):
            DA = []
            error = []
            DA_mc = []
            raw_error_mc = []
            error_mc = []
            for i in range(len(turn_sampling)):
                DA.append(np.power(integrate.romb(values[i][j][::jump], d_preliminar_alpha * jump) * 0.5, 1/4))
                temp = np.power(integrate.romb(values[i][j][::jump * 2], d_preliminar_alpha * jump * 2) * 0.5, 1/4)
                error.append(np.absolute(DA[-1] - temp))
                
                DA_mc.append(np.power(np.average(values[i][j][::jump]), 1/4))
                raw_error_mc.append(np.std(values[i][j][::jump]))
                error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][j][::jump]) / np.sqrt(np.size(values[i][j][::jump])))
                
            DA_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "int")] = DA
            error_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "int")] = error
            DA_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "mc")] = DA_mc
            error_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "mc")] = error_mc
            raw_error_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "mc")] = raw_error_mc

  0%|          | 0/500 [00:00<?, ?it/s]/home/carlidel/Insync/carlo.montanari3@studio.unibo.it/OneDrive Biz/optimized_code/henon_map/henon_map/cpu_henon_core.py:148: RuntimeWarning: invalid value encountered in true_divide
  matrices = np.nansum(matrices.reshape(

  4%|▎         | 18/500 [06:13<2:43:47, 20.39s/it]

## Saving data

In [ ]:
with open("data/DA_2.pkl", 'wb') as f:
    pickle.dump(DA_2, f, protocol=4)
    
with open("data/error_2.pkl", 'wb') as f:
    pickle.dump(error_2, f, protocol=4)

with open("data/raw_error_2.pkl", 'wb') as f:
    pickle.dump(raw_error_2, f, protocol=4)    

# with open("data/count_matrix_2.pkl", 'wb') as f:
#     pickle.dump(count_matrix_2, f, protocol=4)
    
# with open("data/avg_matrix_2.pkl", 'wb') as f:
#     pickle.dump(average_matrix_2, f, protocol=4)

with open("data/DA_5.pkl", 'wb') as f:
    pickle.dump(DA_5, f, protocol=4)
    
with open("data/error_5.pkl", 'wb') as f:
    pickle.dump(error_5, f, protocol=4)

# In the end, we need to plot stuff here!

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
cmap = matplotlib.cm.get_cmap('plasma')

import matplotlib.ticker as ticker

for key in tqdm(list(sorted(count_matrix_2, key=lambda a : a[2]))):
    if key[0] == epsilon:
        for i in tqdm(range(len(average_matrix_2[key]))):
            fig, axs = plt.subplots(1,2, figsize=(12,6))

            coso = axs[0].imshow(np.nanmean(average_matrix_2[key][i], axis=0), origin="lower", extent=(0, np.pi*2, 0, np.pi*2))
            axs[0].set_title("Average radius measured\n$\\varepsilon={}$, $\\alpha$ samples $= {}$, $N$ iters $={}$".format(key[0], key[1], key[2]))
            axs[0].set_xlabel("$\\theta_1$")
            axs[0].set_ylabel("$\\theta_2$")
            fig.colorbar(coso, ax=axs[0])

            axs[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
            axs[0].xaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))
            axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
            axs[0].yaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))

            coso = axs[1].imshow(np.nanmean(count_matrix_2[key][i], axis=0), origin="lower", extent=(0, np.pi*2, 0, np.pi*2))
            axs[1].set_title("Number of samples\n$\\varepsilon={}$, $\\alpha$ samples $= {}$, $N$ iters $={}$".format(key[0], key[1], key[2]))
            axs[1].set_xlabel("$\\theta_1$")
            axs[1].set_ylabel("$\\theta_2$")
            fig.colorbar(coso, ax=axs[1])

            axs[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
            axs[1].xaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))
            axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(int(x/np.pi)) + "$\\pi$"))
            axs[1].yaxis.set_major_locator(ticker.MultipleLocator(base=np.pi))

            plt.tight_layout()
            # print(key)
            plt.savefig("img/cm_eps_" + str(int(key[0])) + "_N_" + str(key[1]) + "_t_" + str(key[2]) + "_el_" + str(i) + ".jpg", dpi=300)
            plt.close()

In [20]:
#import ffmpeg

#name = "img/movie.mp4"
#(
#    ffmpeg
#    .input('img/cm_eps*el*.jpg', pattern_type='glob', framerate=2)
#    .output(name)
#    .run()
#)

(None, None)